In [12]:
import sys 
sys.path.append('..')
from utils.paths import make_dirs, QUARTERS_DICT, HASHTAG_TOP_DIR, HASHTAG_FREQUENCY_DIR_ALL, POTATOES_CLEANED_DIR, FORBIDDEN_HASHTAGS_LIST, POTATOES_HASHTAGS_CLEAN, QUARTER_RESULTS, POTATOES_DIR, QUARTERS_ALL_POSTS, HASHTAGS_PER_USER_DIR, HASHTAG_FREQUENCY_DIR, CONNECTED_COMPONENTS_DIR, CONNECTED_COMPONENTS_PUBLIC_SPACES_DIR, HASHTAG_FREQUENCY_PUBLIC_SPACES_DIR, HASHTAGS_PER_USER_PUBLIC_SPACES_DIR
from utils.utils import load_dataframes, load_dataframe, write_dict_to_csv, write_list_to_csv, write_df_to_csv
import pandas as pd
from frequency.frequency import count_hashtag_frequency, group_by_user
import os

make_dirs()
import numpy as np

min_count = 10

## Count and relative amount per quarter

In [13]:
min_count = 10

dict = QUARTERS_DICT()
for quarter in dict:
    print('>> now processing', quarter)
    df = dict[quarter]
    df = group_by_user(df)
    write_df_to_csv(df, quarter, HASHTAGS_PER_USER_DIR)
    fdist = count_hashtag_frequency(df)
    df = pd.DataFrame(fdist, columns=['hashtag', 'count'])
    df.drop(df[df['count'] < min_count].index, inplace = True)
    count_quarter = df.iloc[0]['count']
    df['relative_amount'] = df['count'] / count_quarter
    write_df_to_csv(df, quarter, HASHTAG_FREQUENCY_DIR)


>> now loading: allach
>> now loading: altperlach
>> now loading: altstadt
>> now loading: amhart
>> now loading: arabellapark
>> now loading: au
>> now loading: aubing
>> now loading: bergamlaim
>> now loading: bogenhausen
>> now loading: borstei
>> now loading: daglfing
>> now loading: denning
>> now loading: fasanerie
>> now loading: fasangarten
>> now loading: feldmoching
>> now loading: forstenried
>> now loading: freiham
>> now loading: freimann
>> now loading: fröttmaning
>> now loading: fürstenried
>> now loading: giesing
>> now loading: hadern
>> now loading: haidhausen
>> now loading: harlaching
>> now loading: harras
>> now loading: harthof
>> now loading: hasenbergl
>> now loading: hellabrunn
>> now loading: herzogpark
>> now loading: isarvorstadt
>> now loading: johanneskirchen
>> now loading: kieferngarten
>> now loading: kirchtrudering
>> now loading: laim
>> now loading: lehel
>> now loading: lerchenau
>> now loading: lochhausen
>> now loading: ludwigsvorstadt
>> now lo

# Count and relative amount for city

In [14]:
min_count = 10

df = pd.DataFrame()
dict = QUARTERS_DICT()
for quarter in dict:
    df = df.append(dict[quarter])
    print('>> now processing', quarter)


df = df.drop_duplicates(subset='post_url', keep="first")
df = group_by_user(df)

user_count = len(df)

fdist = count_hashtag_frequency(df)
df = pd.DataFrame(fdist, columns=['hashtag', 'count'])
df.drop(df[df['count'] < min_count].index, inplace = True)
df['relative_amount'] = df['count'] / user_count


write_df_to_csv(df,'count_all_quarters', HASHTAG_FREQUENCY_DIR_ALL)

>> now loading: allach
>> now loading: altperlach
>> now loading: altstadt
>> now loading: amhart
>> now loading: arabellapark
>> now loading: au
>> now loading: aubing
>> now loading: bergamlaim
>> now loading: bogenhausen
>> now loading: borstei
>> now loading: daglfing
>> now loading: denning
>> now loading: fasanerie
>> now loading: fasangarten
>> now loading: feldmoching
>> now loading: forstenried
>> now loading: freiham
>> now loading: freimann
>> now loading: fröttmaning
>> now loading: fürstenried
>> now loading: giesing
>> now loading: hadern
>> now loading: haidhausen
>> now loading: harlaching
>> now loading: harras
>> now loading: harthof
>> now loading: hasenbergl
>> now loading: hellabrunn
>> now loading: herzogpark
>> now loading: isarvorstadt
>> now loading: johanneskirchen
>> now loading: kieferngarten
>> now loading: kirchtrudering
>> now loading: laim
>> now loading: lehel
>> now loading: lerchenau
>> now loading: lochhausen
>> now loading: ludwigsvorstadt
>> now lo

# GET uniqueness and rank per quarter

In [15]:
import math

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

def get_uniqueness(dfs, df_all, out_dir):
    idx = 1

    for quarter in dfs:
        print ('file no:', idx)
        idx += 1
        print ('>> now processing:', quarter)

        df = dfs[quarter]
        df.set_index('hashtag')

        df = df.join(df_all.set_index('hashtag'), on='hashtag', lsuffix='_quarter', rsuffix='_city')
        df['uniqueness'] = df['relative_amount_quarter'] / df['relative_amount_city']
        df['uniqueness_log'] = np.log2(df['uniqueness'])

        df['opacitysigmoidlog'] = df['uniqueness_log'].apply(sigmoid)
        df['opacitytanh'] = np.tanh(df['uniqueness'])/(np.tanh(1)*2)



        df.sort_values(by=['uniqueness'], ascending=False, inplace=True)

        write_df_to_csv(df, quarter, out_dir)

        # TRY THE OPACITY THINGS WITH NP

        df['rank'] = df['uniqueness_log']*df['count_quarter']
        df.sort_values(by=['rank'], ascending=False, inplace=True)
        write_df_to_csv(df, quarter + '_rank', out_dir)

    return

In [16]:
dfs = load_dataframes(HASHTAG_FREQUENCY_DIR)
df_all = load_dataframe(HASHTAG_FREQUENCY_DIR_ALL, 'count_all_quarters')

get_uniqueness(dfs, df_all, HASHTAG_TOP_DIR)

>> now loading: allach
>> now loading: altperlach
>> now loading: altstadt
>> now loading: amhart
>> now loading: arabellapark
>> now loading: au
>> now loading: aubing
>> now loading: bergamlaim
>> now loading: bogenhausen
>> now loading: borstei
>> now loading: daglfing
>> now loading: denning
>> now loading: fasanerie
>> now loading: fasangarten
>> now loading: feldmoching
>> now loading: forstenried
>> now loading: freiham
>> now loading: freimann
>> now loading: fröttmaning
>> now loading: fürstenried
>> now loading: giesing
>> now loading: hadern
>> now loading: haidhausen
>> now loading: harlaching
>> now loading: harras
>> now loading: harthof
>> now loading: hasenbergl
>> now loading: hellabrunn
>> now loading: herzogpark
>> now loading: isarvorstadt
>> now loading: johanneskirchen
>> now loading: kieferngarten
>> now loading: kirchtrudering
>> now loading: laim
>> now loading: lehel
>> now loading: lerchenau
>> now loading: lochhausen
>> now loading: ludwigsvorstadt
>> now lo